In [1]:
!pip install pyaudio

In [2]:
!pip install wave

  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1220 sha256=a3e022fec820c79ae36ccf1796cedd1b81077a2868f9e13b3f8b74b0e9677fa8
  Stored in directory: c:\users\shweta\appdata\local\pip\cache\wheels\ee\3e\b3\bdee8d885ec04fd42bd239e9d9ab9dc94e69aad318c0d9936c
Successfully built wave


In [3]:
import pyaudio
import wave

CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2 
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "output10.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [15]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

In [5]:
pip install librosa

  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23703 sha256=6e841419b81e24c4a4d079732d0fdbbcebc214babd8478e718152a5465209902
  Stored in directory: c:\users\shweta\appdata\local\pip\cache\wheels\e4\76\a4\cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pad_sequences from keras.preprocessing.sequence

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement from (from versions: none)
ERROR: No matching distribution found for from


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
from keras import regularizers

In [17]:
import os

In [42]:
mylist= os.listdir('C:/Users/Shweta/Downloads/NLP')

In [43]:
type(mylist)

list

In [46]:
mylist[200]

'03-01-02-02-01-01-09.wav'

In [24]:
!pip install soundfile

In [25]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

In [26]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result
 

In [30]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
          y.append(l[emotion])
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [31]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [32]:
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [37]:
dataset_path = 'https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio'

In [33]:
# Function to extract features from each audio file
def extract_features(file_path):
    audio, sampling_rate = librosa.load(file_path, sr=22050, duration=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=30)
    features = np.mean(mfccs.T, axis=0)
    return features

# Function to extract features from each audio file without silent moments
def extract_features_trim(file_path):
    audio, sampling_rate = librosa.load(file_path, sr=22050, duration=None)
    audio, index = librosa.effects.trim(audio, top_db=20, frame_length=512, hop_length=256)
    mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=30)
    features = np.mean(mfccs.T, axis=0)
    return features

# Function to preprocess the data
def preprocess_data(dataset_path):
    X = []
    y = []
    for folder in os.listdir(dataset_path):
        for file in os.listdir(os.path.join(dataset_path, folder)):
            if file.endswith('.wav'):
                
                emotion = file.split('-')[2]
                if int(emotion) == 1:
                    label = 'Neutral'
                elif int(emotion) == 2:
                    label = 'Calm'
                elif int(emotion) == 3:
                    label = 'Happy'
                elif int(emotion) == 4:
                    label = 'Sad'
                elif int(emotion) == 5:
                    label = 'Angry'
                elif int(emotion) == 6:
                    label = 'Fearful'
                elif int(emotion) == 7:
                    label = 'Disgust'
                elif int(emotion) == 8:
                    label = 'Surprised'
                else:
                    label = 'UNK'
                    
                file_path = os.path.join(dataset_path, folder, file)
                features = extract_features(file_path)
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)

# Function for displaying confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4, 3))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix For {}'.format(title))
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

In [38]:
# Preprocess the data
X, y = preprocess_data(dataset_path)

# Print the shape of the feature matrix and the label array
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio'

In [40]:
os.getcwd()

'C:\\Users\\Shweta'

In [41]:
os.listdir()

['.arduinoIDE',
 '.cache',
 '.conda',
 '.condarc',
 '.continuum',
 '.eclipse',
 '.idlerc',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.keras',
 '.matplotlib',
 '.opera',
 '.p2',
 '.spyder-py3',
 '.tooling',
 '.VirtualBox',
 '.vscode',
 '.Xilinx',
 '3D Objects',
 'ADA1.ipynb',
 'ADA2.ipynb',
 'AdaClassificationlab.ipynb',
 'Adalab.csv',
 'ADAlab_Project-Copy1.ipynb',
 'ADAlab_Project.ipynb',
 'all-data.csv',
 'anaconda3',
 'AppData',
 'Application Data',
 'Assignment-2 nd 4 NLP.ipynb',
 'auction.ipynb',
 'awerdt45',
 'BullseyeCoverageError.txt',
 'car_evaluation.csv',
 'catboost_info',
 'Contacts',
 'Cookies',
 'Corona_NLP_test.csv',
 'Customer_Segmentation.ipynb',
 'Desktop',
 'Documents',
 'Downloads',
 'eclipse',
 'eclipse-workspace',
 'EcoPreprocessed.csv',
 'Favorites',
 'FinalAdalab - anemia (1).csv',
 'IntelGraphicsProfiles',
 'Jedi',
 'LAB 2.ipynb',
 'Lab-1.v',
 'Links',
 'Litdac.csv',
 'Local Settings',
 'Mall_Customers.csv',
 'ml project.ipynb',
 'MLProject_L3-4.ipynb'